In [ ]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.layers import Input, LSTM, Embedding, Dense, \
    Concatenate, TimeDistributed, RepeatVector, Attention
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.losses import mse

In [ ]:
import pickle
from keras.models import load_model
tokenizer = pickle.load(open('/content/tokenizer.pickle', 'rb'))
model = load_model('/content/meddoc_final.h5')


In [ ]:
#Same order
from nltk.tokenize import sent_tokenize
import numpy as np
from sklearn.cluster import KMeans
from keras.models import load_model
def generate_summary_with_clustering(text, model_path, num_clusters, num_sentences):
    # Load the trained model
    model = load_model(model_path)
    max_text_len = 5000

    # Tokenize the input text
    input_sequence = tokenizer.texts_to_sequences([text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_text_len, padding='post')

    # Define a new model to get the hidden state
    encoder_inputs = model.input
    encoder_outputs, state_h, state_c = model.layers[2].output
    encoder_states = [state_h, state_c]
    encoder_model = Model(encoder_inputs, encoder_states)

    # Use the encoder to get the hidden state
    hidden_state = encoder_model.predict(input_sequence)
    hidden_state = np.reshape(hidden_state, (1, -1))

    # Tokenize the sentences in the input text and keep track of their indices
    sentences = sent_tokenize(text)
    sentence_indices = list(range(len(sentences)))

    # Use k-means clustering to group the sentences based on their similarity to the hidden state
    sentence_vectors = []
    for sentence in sentences:
        sentence_sequence = tokenizer.texts_to_sequences([sentence])
        sentence_sequence = pad_sequences(sentence_sequence, maxlen=max_text_len, padding='post')
        sentence_vector = encoder_model.predict(sentence_sequence)
        sentence_vectors.append(sentence_vector)

    sentence_vectors = np.array(sentence_vectors)
    kmeans = KMeans(n_clusters=num_clusters, n_init=10, random_state=0).fit(
        sentence_vectors.reshape(len(sentence_vectors), -1)
    )

    # Get the sentences from the most similar cluster centers in the order of their indices
    summary_sentence_indices = set()
    summary_sentences = []
    for i in range(num_clusters):
        cluster_sentence_indices = [sentence_indices[j] for j in np.where(kmeans.labels_ == i)[0]]
        if cluster_sentence_indices:
            closest_center_index = np.argmin(np.linalg.norm(hidden_state - kmeans.cluster_centers_[i], axis=1))
            closest_sentence_index = cluster_sentence_indices.pop(closest_center_index)
            cluster_sentence_indices = [idx for idx in cluster_sentence_indices if idx not in summary_sentence_indices]
            cluster_sentence_indices = sorted(cluster_sentence_indices)
            cluster_sentence_indices.insert(0, closest_sentence_index)
            cluster_sentences = [sentences[j] for j in cluster_sentence_indices]
            summary_sentence_indices.update(cluster_sentence_indices)
            summary_sentences.extend(cluster_sentences)
            if len(summary_sentence_indices) >= num_sentences:
                break


    summary = ' '.join(summary_sentences)

    return summary


In [ ]:
text='Sex: F Service: UROLOGY Allergies: Patient recorded as having No Known Allergies to Drugs Attending: . Chief Complaint: renal mass Major Surgical or Invasive Procedure: right laparascopic radical nephrectomy Dr. Dr. History of Present Illness: y/o healthy female with incidental finding of right renal mass suspicious for RCC following MRI on . Past Medical History: PMH: nonspecific right axis deviation PSH cesarean section ALLNKDA Social History: Family History: no history of RCC Pertinent Results: 07:15AM BLOOD WBC7.6 RBC3.82 Hgb11.9 Hct33.8 MCV89 MCH31.2 MCHC35.2 RDW12.8 Plt 07:15AM BLOOD Glucose150 UreaN10 Creat0.9 Na138 K3.8 Cl104 HCO327 AnGap11 Brief Hospital Course: Patient was admitted to Urology after undergoing laparoscopic right radical nephrectomy. No concerning intraoperative events occurred please see dictated operative note for details. The patient received perioperative antibiotic prophylaxis. The patient was transferred to the floor from the PACU in stable condition. On POD0 pain was well controlled on PCA hydrated for urine output 30cc/hour provided with pneumoboots and incentive spirometry for prophylaxis and ambulated once. On POD1foley was removed without difficulty basic metabolic panel and complete blood count were checked pain control was transitioned from PCA to oral analgesics diet was advanced to a clears/toast and crackers diet. On POD2 diet was advanced as tolerated. The remainder of the hospital course was relatively unremarkable. The patient was discharged in stable condition eating well ambulating independently voiding without difficulty and with pain control on oral analgesics. On exam incision was clean dry and intact with no evidence of hematoma collection or infection. The patient was given explicit instructions to followup in clinic with in 3 weeks. Medications on Admission: none Discharge Medications: 1. HydrocodoneAcetaminophen mg Tablet Sig: Tablets PO Q6H every 6 hours as needed for break through pain only score 4 . Disp:60 Tablets Refills:0 2. Docusate Sodium 100 mg Capsule Sig: One 1 Capsule PO BID 2 times a day. Disp:60 Capsules Refills:2 Discharge Disposition: Home Discharge Diagnosis: renal cell carcinoma Discharge Condition: stable Discharge Instructions: You may shower but do not bathe swim or immerse your incision. Do not eat constipating foods for weeks drink plenty of fluids Do not lift anything heavier than a phone book 10 pounds or drive until you are seen by your Urologist in followup Tylenol should be used as your first line pain medication. If your pain is not well controlled on Tylenol you have been prescribed a narcotic pain medication. Use in place of Tylenol. Do not exceed 4 gms of Tylenol in total daily Do not drive or drink alcohol while taking narcotics Resume all of your home medications except hold NSAID aspirin advil motrin ibuprofen until you see your urologist in followup If you have fevers 101.5 F vomiting or increased redness swelling or discharge from your incision call your doctor or go to the nearest ER Call Dr. to set up followup appointment and if you have any urological questions. Followup Instructions:'

In [ ]:
text='Sex: F Service: UROLOGY Allergies: Celexa Attending: . Chief Complaint: Adrenal Mass Major Surgical or Invasive Procedure: 1. Laparoscopic right adrenalectomy with removal of large adrenal mass 12cm . 2. Percutaneous liver biopsy. History of Present Illness: yo F with right 10 cm nonfunctional adrenal mass. Past Medical History: Past medical history of hypertension denies history of MI or stroke. 3 vaginal deliveries appendectomy Social History: Family History: Family history is negative for adrenal issues Physical Exam: WdWn NAD AVSS Interactive cooperative Abdomen soft appropriately tender along incisions Incisions otherwise c/d/i Extremities w/out edema or pitting and there is no reported calf pain to deep palpation Pertinent Results: 12:50PM BLOOD WBC 6.4 RBC 2.72 Hgb 7.6 Hct 23.8 MCV 88 MCH 27.9 MCHC 31.9 RDW 15.8 RDWSD 50.2 Plt 12:50PM BLOOD Glucose 91 UreaN 7 Creat 0.6 Na 137 K 4.1 Cl 103 HCO3 23 AnGap 11 12:50PM BLOOD Calcium 7.7 Mg 1.7 Brief Hospital Course: Ms. was admitted to Urology with right adrenal mass and underwent laparoscopic right adrenalectomy with percutaneous liver biopsy. No concerning intraoperative events occurred please see dictated operative note for details. The patient received perioperative antibiotic prophylaxis. The patient was transferred to the floor from the PACU in stable condition. On POD0 pain was well controlled on PCA hydrated for urine output 30cc/hour provided with pneumoboots and incentive spirometry for prophylaxis and ambulated once. On POD1 the patient was restarted on home medications basic metabolic panel and complete blood count were checked pain control was transitioned from PCA to oral analgesics diet was advanced to a clears/toast and crackers diet. Urethral Foley catheter was removed without difficulty and on POD2 diet was advanced as tolerated. The remainder of the hospital course was relatively unremarkable. The patient was discharged in stable condition eating well ambulating independently voiding without difficulty and with pain control on oral analgesics. On exam incision was clean dry and intact with no evidence of hematoma collection or infection. The patient was given explicit instructions to follow up in clinic in four weeks. Medications on Admission: The Preadmission Medication list is accurate and complete. 1. lisinopril hydrochlorothiazide mg oral DAILY 2. Omeprazole 20 mg PO DAILY 3. raloxifene 60 mg oral DAILY 4. Simvastatin 20 mg PO QPM 5. Citracal D3 calcium phos calcium phosphate vitamin D3 250 mg calcium 500 unit oral DAILY 6. Vitamin D 1000 UNIT PO DAILY 7. Centrum Silver Women multivit min iron FA lutein 8 mg iron 400 mcg 300 mcg oral DAILY Discharge Medications: 1. Acetaminophen 650 mg PO Q6H:PRN Pain Mild 2. Ibuprofen 600 mg PO Q8H:PRN Pain Mild RX ibuprofen 600 mg ONE tablet s by mouth Q8hrs Disp 25 Tablet Refills: 0 3. OxyCODONE Immediate Release 2.5 5 mg PO Q4H:PRN Pain Moderate RX oxycodone 5 mg HALF to ONE FULL tablet by mouth Q4hrs Disp 30 Tablet Refills: 0 4. Senna 8.6 mg PO BID Duration: 4 Doses RX sennosides Senokot 8.6 mg ONE tab by mouth x daily Disp 30 Tablet Refills: 0 5. Centrum Silver Women multivit min iron FA lutein 8 mg iron 400 mcg 300 mcg oral DAILY 6. Citracal D3 calcium phos calcium phosphate vitamin D3 250 mg calcium 500 unit oral DAILY 7. lisinopril hydrochlorothiazide mg oral DAILY 8. Omeprazole 20 mg PO DAILY 9. raloxifene 60 mg oral DAILY 10. Simvastatin 20 mg PO QPM 11. Vitamin D 1000 UNIT PO DAILY Discharge Disposition: Home Discharge Diagnosis: PREOPERATIVE DIAGNOSIS: Right adrenal mass. POSTOPERATIVE DIAGNOSIS: Right adrenal mass with possible liver fibrosis. Discharge Condition: Mental Status: Clear and coherent. Level of Consciousness: Alert and interactive. Activity Status: Ambulatory Independent. Discharge Instructions: Please also refer to the provided handout that details instructions and expectations for your post operative phase as made available by your urologist. Resume your pre admission/home medications except as noted. ALWAYS call to inform review and discuss any medication changes and your post operative course with your primary care doctor. reduce the strain/pressure on your abdomen and incision sites remember to log roll onto your side and then use your hands to push yourself upright while taking advantage of the momentum of putting your legs/feet to the ground. There may be bandage strips called steristrips which have been applied to reinforce wound closure. Allow these bandage strips to fall off on their own over time but PLEASE REMOVE ANY REMAINING GAUZE DRESSINGS WITHIN 2 DAYS OF DISCHARGE. You may get the steristrips wet. UNLESS OTHERWISE NOTED AVOID aspirin or aspirin containing products and supplements that may have blood thinning effects like Fish Oil Vitamin E etc. . This will be noted in your medication reconciliation. IF PRESCRIBED see the MEDICATION RECONCILIATION : IBUPROFEN the ingredient of Advil Motrin etc. may be taken even though you may also be taking Tylenol/Acetaminophen. You may alternate these medications for pain control. For pain control try TYLENOL acetaminophen FIRST then ibuprofen and then take the narcotic pain medication as prescribed if additional pain relief is needed. Ibuprofen should always be taken with food. Please discontinue taking and notify your doctor should you develop blood in your stool dark tarry stools Call your Urologist s office to schedule/confirm your follow up appointment in 4 weeks AND if you have any questions. Do not eat constipating foods for weeks drink plenty of fluids to keep hydrated No vigorous physical activity or sports for 4 weeks or until otherwise advised. Light household chores/activity and leisurely walking/activity is OK and should be continued. Do NOT be a couch potato Tylenol should be your first line pain medication. A narcotic pain medication has been prescribed for breakthrough pain . Max daily Tylenol acetaminophen dose is THREE to FOUR grams from ALL sources AVOID lifting/pushing/pulling items heavier than 10 pounds or 3 kilos about a gallon of milk or participate in high intensity physical activity which includes intercourse until you are cleared by your Urologist in follow up. No DRIVING for THREE WEEKS or until you are cleared by your Urologist You may shower normally but do NOT immerse your incisions or bathe Do not drive or drink alcohol while taking narcotics and do not operate dangerous machinery You may be given prescriptions for a stool softener and/or a gentle laxative. These are over the counter medications that may be health care spending account reimbursable. Colace docusate sodium may have been prescribed to avoid post surgical constipation or constipation related to use of narcotic pain medications. Discontinue if loose stool or diarrhea develops. Colace is a stool softener NOT a laxative. Senokot or any gentle laxative may have been prescribed to further minimize your risk of constipation. If you have fevers 101.5 F vomiting or increased redness swelling or discharge from your incision call your doctor or go to the nearest emergency room. Followup Instructions:'

In [ ]:
text="Sex: F Service: MEDICINE Allergies: Attending: Chief Complaint: HTN leg edema Major Surgical or Invasive Procedure: none History of Present Illness: Mrs. is a year old female with PMH of hypertension and Hep C who presents with 1 month edema and HTN to SBP 215 at s office. She was previously prescribed 20mg lasix for edema unclear if she was taking it. No DOE orthopnea CP cough palpitations headache visual changes n/v. She was given 80mg lasix IV in the ED but no BP meds. Today she reports feeling fine other than having to urinate incessantly. She also complains of constipation. Past Medical History: Hypertension Hepatitis C Social History: Family History: Noncontributory Physical Exam: Admission physical exam: Vitals: Tc 98 Tm 98 BP 170/89 158 188 systolic HR 59 R 20 O2 97RA Tele: overnight sinus brady as low as 38 mostly in I/O PMN: General: WDWN female appearing younger than her stated age Alert oriented no acute distress. HEENT: Sclera anicteric MMM oropharynx clear Neck: supple JVP 8cm H2O no LAD Lungs: breathing unlabored clear to auscultation bilaterally no wheezes rales ronchi CV: bradycardic regular rhythm systolic murmur at RUSB diastolic murmur at apex S3 heard at apex. Abdomen: soft non tender non distended bowel sounds present no rebound tenderness or guarding no organomegaly Ext: Warm well perfused 2 pulses no clubbing cyanosis. 2 pitting edema b/l and equal in lower extremities to knee Neuro: AOx3 CNII XII grossly intact no focal deficits Discharge physical exam: VS: 98.4 156/88 62 18 100 ra I/O: not recorded daily weight not recorded General: WDWN female appearing younger than her stated age Alert oriented no acute distress. HEENT: Sclera anicteric MMM oropharynx clear Neck: supple JVP 8cm H2O no LAD Lungs: breathing unlabored clear to auscultation bilaterally no wheezes rales ronchi CV: RRR systolic murmur at RUSB diastolic murmur at apex S3 heard at apex. Abdomen: soft non tender non distended bowel sounds present no rebound tenderness or guarding no organomegaly Ext: Warm well perfused 2 pulses no clubbing cyanosis. 2 pitting edema b/l and equal in lower extremities to knee Neuro: AOx3 CNII XII grossly intact no focal deficits Pertinent Results: Admission labs: 07:58PM BLOOD WBC 3.2 RBC 4.62 Hgb 12.8 Hct 41.4 MCV 90 MCH 27.8 MCHC 31.0 RDW 14.8 Plt 07:58PM BLOOD Neuts 63 Bands 0 Monos 7 Eos 2 Baso 0 Atyps 2 Myelos 0 07:58PM BLOOD Hypochr OCCASIONAL Anisocy OCCASIONAL Poiklo NORMAL Macrocy NORMAL Microcy NORMAL Polychr NORMAL 07:58PM BLOOD Glucose 85 UreaN 13 Creat 1.0 Na 142 K 4.1 Cl 105 HCO3 28 AnGap 13 05:00AM BLOOD LD LDH 440 02:35PM BLOOD CK CPK 181 07:58PM BLOOD proBNP 4362 07:20PM BLOOD CK MB 6 cTropnT 0.01 10:47AM BLOOD CK MB 5 cTropnT 0.03 02:35PM BLOOD CK MB 4 cTropnT 0.03 05:00AM BLOOD Calcium 9.6 Phos 2.5 Mg 2.1 Discharge labs: 07:33AM BLOOD WBC 3.6 RBC 4.40 Hgb 12.4 Hct 38.4 MCV 87 MCH 28.1 MCHC 32.1 RDW 14.8 Plt 07:33AM BLOOD Glucose 98 UreaN 18 Creat 0.9 Na 141 K 4.3 Cl 108 HCO3 28 AnGap 9 07:33AM BLOOD Calcium 9.1 Phos 3.3 Mg 2.4 Pertinent micro/path: none Pertinent imaging: CXR : PA and lateral views of the chest are compared to previous exam from . The lungs are clear of consolidation effusion or pulmonary vascular congestion. Cardiac silhouette is enlarged but unchanged. There is marked tortuosity of the descending thoracic aorta which is unchanged from prior. Osseous and soft tissue structures are unremarkable. IMPRESSION: No acute cardiopulmonary process. Cardiomegaly and marked tortuosity of the descending thoracic aorta unchanged. Brief Hospital Course: F c/o progressive lower extremity pitting edema x 1mo with hypertension to SBP 200 today. Patient likely with acute diastolic CHF. Acute on chronic diastolic CHF: Patient with 1 month of worsening edema found to be hypertensive and possibly noncompliant with medications history varied . Hx of LVH with preserved EF seen on last echo so likely diastolic HF. She was diuresed fairly aggressively with 80mg IV lasix in the ED just prior to transfer to the medical floor. On the floor she had diuresed approx 2L by from and was continuing to urinate frequently. Later that morning she developed a fib with RVR which resolved with 500cc IVF bolus and 5mg lopressor IV. Further diuresis was held and she no longer had episodes of a fib with RVR. As for a cause of her acute on chronic heart failure trops were drawn to rule out ischemia. They were mildly elevated with a flat MB possibly reflecting demand ischemia or related to . More likely her acute exacerbation was due to uncontrolled hypertension. Her edema improved with diuresis. She was discharged on her home dose of lasix amlodipine and moexipril. She was unable to have a TTE during her stay due to limited availability of technicians but it was not thought to be crucial to her acute management. Recommend repeat echo as an outpatient given her new exacerbation and that her last one was in . A fib with RVR: Scenario as described above. Pt has no recorded history of AF. CHADS2 score of 3 CHF HTN age . Since pt s AF resolved without recurrence and given her age it was decided not to anticoagulate her with warfarin. Instead she was started on ASA daily. HTN: patient with severe hypertension SBP 215 on presentation to the ED. She was asymptomatic and had no signs of end organ damage. Her SBP improved to the 140s with diuresis and administration of her home medications. Pt may have difficulty remembering to take her medications everyday and set her up for home services. constipation: resolved with bowel regimen. Pt discharged on colace senna and miralax. Hepatitis C: pt not on suppressive therapy. Leukopenia: 2 atypicals on diff. possibly due to HCV vs MDS. LDH wnl so not likely a scenario of high cell turnover. Recommend further workup as outpatient. Transitional issues: Repeat TTE as outpatient Medications on Admission: Amlodipine 10mg daily furosemide 20mg daily moexipril 30mg daily Discharge Medications: 1. Amlodipine 10 mg PO DAILY 2. Aspirin 325 mg PO DAILY RX aspirin 325 mg 1 tablet s by mouth daily Disp 30 Tablet Refills: 0 3. Docusate Sodium 100 mg PO BID RX docusate sodium 100 mg 1 capsule s by mouth twice a day Disp 60 Capsule Refills: 0 4. Furosemide 20 mg PO QD 5. Milk of Magnesia 30 mL PO BID constipation 6. Moexipril 30 mg PO DAILY 7. Polyethylene Glycol 17 g PO 1X Duration: 1 Doses RX polyethylene glycol 3350 Miralax 17 gram/dose 1 scoop by mouth daily Disp 1 Bottle Refills: 0 8. Senna 1 TAB PO BID:PRN constipation RX sennosides senna 8.6 mg 1 tab by mouth twice a day Disp 60 Tablet Refills: 0 Discharge Disposition: Home Discharge Diagnosis: Primary Hypertension uncontrolled acute on chronic diastolic congestive heart failure Atrial fibrillation Secondary Hepatitis C virus Leukopenia Discharge Condition: Mental Status: Clear and coherent. Level of Consciousness: Alert and interactive. Activity Status: Ambulatory Independent. Discharge Instructions: Dear Ms. It was a pleasure caring for you at . You were admitted for high blood pressure and extra fluid in your legs. We gave you a medicine called a diuretic to help reduce your blood pressure and the extra fluid. You also developed a heart rhythm problem called atrial fibrillation. Because of this you will need to take aspirin daily to prevent blood clots. We recommend that you have an echocardiogram of your heart in the future to look for any new findings that could be causing your heart problems. We also prescribed you medications for constipation. You may take these as needed. We made the following changes to your medications: START aspirin 325mg daily START docusate START senna START miralax Please continue to take your other medications as prescribed. Be sure to take them on time EVERYDAY. Followup Instructions:"

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
model_path='/content/meddoc_final.h5'
num_clusters=50
num_sentences=30
summary=generate_summary_with_clustering(text, model_path, num_clusters, num_sentences)

1/1 [==============================] - 0s 24ms/step


In [ ]:
print(summary)

He is not sure why he is here and begins to tell me about a party in his house with a motor cross race in his backyard. NEUROLOGIC: AOx3. cogwheel UE b/l. NEUROLOGIC: AOx3. cogwheel UE b/l. He successfully uptitrated the medicine to 0.75/0.625/0.625 but began to have visual hallucinations and confusion so on his neurologist recommended decreasing the dose to 0.625 TID. Pertinent Results: ADMISSION LABS: 10:30PM BLOOD WBC8.6 RBC4.03 Hgb12.8 Hct38.2 MCV95 MCH31.8 MCHC33.5 RDW13.0 RDWSD45.2 Plt 10:30PM BLOOD Neuts48.1 Monos15.9 Eos2.0 Baso0.6 Im AbsNeut4.13 AbsLymp2.86 AbsMono1.36 AbsEos0.17 AbsBaso0.05 10:30PM BLOOD PTT23.4 10:30PM BLOOD Glucose100 UreaN17 Creat0.8 Na132 K5.8 Cl98 HCO319 AnGap15 10:30PM BLOOD ALT18 AST38 AlkPhos39 TotBili0.4 10:30PM BLOOD Lipase47 10:30PM BLOOD cTropnT0.01 10:30PM BLOOD Albumin4.0 Calcium9.9 Phos3.7 Mg2.0 10:30PM BLOOD VitB12570 10:30PM BLOOD TSH1.4 07:00AM BLOOD Trep AbNEG 10:30PM BLOOD ASANEG EthanolNEG AcetmnpNEG TricyclNEG IMAGING: Imaging CT HEAD W/